# Data Cleaning and Exploratory Data Analysis

Important note: The 1 million rows dataset on spotify data DOES NOT have genre information. At least when checked most recently... Idk how this was overlooked, but basically since the 1 million songs dataset does not have any genre information, we cannot use that dataset to make analyses on genre or combine with the billboard charts dataset.

In [26]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns

In [27]:
spotify = pd.read_csv("../data/spotify_tracks.csv")
billboard = pd.read_csv("../data/Hot 100.csv")

In [28]:
hot_100.head(5)

,chart_position,chart_date,song,performer,song_id,instance,time_on_chart,consecutive_weeks,previous_week,peak_position,worst_position,chart_debut,chart_url
0,84,1990-05-05,"""B"" Girls",Young And Restless,"""B"" GirlsYoung And Restless",1,1,1,NaN,84,84,1990-05-05,https://www.billboard.com/charts/hot-100/1990-...
1,78,1990-05-12,"""B"" Girls",Young And Restless,"""B"" GirlsYoung And Restless",1,2,2,84.0,78,84,1990-05-05,https://www.billboard.com/charts/hot-100/1990-...
2,68,1990-05-19,"""B"" Girls",Young And Restless,"""B"" GirlsYoung And Restless",1,3,3,78.0,68,84,1990-05-05,https://www.billboard.com/charts/hot-100/1990-...
3,60,1990-05-26,"""B"" Girls",Young And Restless,"""B"" GirlsYoung And Restless",1,4,4,68.0,60,84,1990-05-05,https://www.billboard.com/charts/hot-100/1990-...
4,58,1990-06-02,"""B"" Girls",Young And Restless,"""B"" GirlsYoung And Restless",1,5,5,60.0,58,84,1990-05-05,https://www.billboard.com/charts/hot-100/1990-...


In [29]:
spotify.head(5)

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [30]:
# Remove duplicates by track_name and keep the first occurrence
genre = spotify.drop_duplicates(subset="track_name")[["track_name", "track_genre"]]

# Left join to keep all billboard songs and bring in genre info
billboard_genre = pd.merge(billboard, genre, how="left", left_on="song", right_on="track_name")

In [31]:
# Distinct songs in billboard
distinct_billboard = billboard.drop_duplicates(subset="song")

# Filter rows where track_genre is not missing (i.e., not NA)
distinct_billboard_genre = billboard_genre[billboard_genre["track_genre"].notna()]


In [34]:
# Extract year from chart_date and count songs per year
distinct_billboard_genre["chart_date"] = pd.to_datetime(distinct_billboard_genre["chart_date"])
distinct_billboard_genre["year"] = distinct_billboard_genre["chart_date"].dt.year

songs_per_year = (
    distinct_billboard_genre
    .drop_duplicates(subset="song")
    .groupby("year")
    .size()
    .reset_index(name="count")
    .sort_values("year", ascending=False)
)
songs_per_year


C:\Users\natth\AppData\Local\Temp\ipykernel_48960\1742980955.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distinct_billboard_genre["chart_date"] = pd.to_datetime(distinct_billboard_genre["chart_date"])
C:\Users\natth\AppData\Local\Temp\ipykernel_48960\1742980955.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distinct_billboard_genre["year"] = distinct_billboard_genre["chart_date"].dt.year


,year,count
67,2025,49
66,2024,79
65,2023,60
64,2022,148
63,2021,133
...,...,...
4,1962,50
3,1961,78
2,1960,62
1,1959,47


In [35]:
# Count distinct songs by genre
songs_per_genre = (
    distinct_billboard_genre
    .drop_duplicates(subset="song")
    ["track_genre"]
    .value_counts()
    .reset_index()
    .rename(columns={"index": "track_genre", "track_genre": "count"})
)
songs_per_genre

,count,count
0,acoustic,221
1,country,170
2,dance,164
3,blues,158
4,alt-rock,140
...,...,...
106,samba,2
107,reggaeton,2
108,indie,2
109,forro,2


In [ ]:
#potentially use this or give up tbh

# spotify_genre_lookup.py
import os
import time
import pandas as pd
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm import tqdm

# ---------- CONFIG ----------
# set these in your environment or fill here (env is safer)
# os.environ["SPOTIPY_CLIENT_ID"] = "YOUR_CLIENT_ID"
# os.environ["SPOTIPY_CLIENT_SECRET"] = "YOUR_CLIENT_SECRET"

# initialize Spotipy client (Client Credentials flow)
client_credentials_manager = SpotifyClientCredentials()
sp = Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=10, retries=3)

# files / data frames (adapt paths)
billboard_df = pd.read_csv("data/hot100.csv")    # must contain 'song' and ideally 'artist' columns
# Optional: If you only have song names, still proceed, but include artist when available.

# Create search queries
def make_query(title, artist=None):
    # include quotes to encourage exact title match; artist helps disambiguate
    if pd.isna(artist) or artist == "":
        return f'track:"{title}"'
    return f'track:"{title}" artist:"{artist}"'

# normalize helper
def normalize_string(s):
    if pd.isna(s): return ""
    return " ".join(str(s).lower().strip().split())

# caching helpers
CACHE_FILE = "spotify_genre_cache.parquet"  # persistent cache between runs

if os.path.exists(CACHE_FILE):
    cache = pd.read_parquet(CACHE_FILE)
else:
    cache = pd.DataFrame(columns=["title", "artist", "track_id", "artist_ids", "artist_genres", "match_score", "spotify_found"])

# quick helper to save cache
def save_cache():
    cache.to_parquet(CACHE_FILE, index=False)

# main lookup
results = []

# iterate unique songs (optionally include artist if present)
# reduce to unique pairs for fewer API calls
if "artist" in billboard_df.columns:
    unique_pairs = billboard_df[["song", "artist"]].drop_duplicates().reset_index(drop=True)
else:
    unique_pairs = billboard_df[["song"]].drop_duplicates().reset_index(drop=True)
    unique_pairs["artist"] = ""  # empty artist

for _, row in tqdm(unique_pairs.iterrows(), total=len(unique_pairs)):
    title = str(row["song"])
    artist = str(row["artist"]) if "artist" in row else ""

    qtitle = normalize_string(title)
    qartist = normalize_string(artist)

    # skip if cached
    cached = cache[(cache["title"] == qtitle) & (cache["artist"] == qartist)]
    if not cached.empty:
        results.append(cached.iloc[0].to_dict())
        continue

    query = make_query(title, artist if artist else None)

    # try search
    try:
        r = sp.search(q=query, type="track", limit=5)
    except Exception as e:
        print("Spotify search failed, sleeping and retrying:", e)
        time.sleep(5)
        try:
            r = sp.search(q=query, type="track", limit=5)
        except Exception as e2:
            print("Retry failed:", e2)
            r = {"tracks": {"items": []}}

    items = r.get("tracks", {}).get("items", [])

    if not items:
        # fallback: try searching title only (broader)
        try:
            r2 = sp.search(q=f'track:"{title}"', type="track", limit=10)
            items = r2.get("tracks", {}).get("items", [])
        except Exception:
            items = []

    if not items:
        # nothing found
        entry = {
            "title": qtitle,
            "artist": qartist,
            "track_id": None,
            "artist_ids": None,
            "artist_genres": None,
            "match_score": 0,
            "spotify_found": False
        }
        cache = pd.concat([cache, pd.DataFrame([entry])], ignore_index=True)
        results.append(entry)
        save_cache()
        continue

    # pick the best candidate heuristically (prefer exact artists if available)
    chosen = None
    for it in items:
        # examine track name and artist names
        track_name = normalize_string(it["name"])
        artist_names = [normalize_string(a["name"]) for a in it["artists"]]
        # simple heuristics:
        if track_name == qtitle and (qartist == "" or qartist in artist_names):
            chosen = it
            break
    if chosen is None:
        # fallback to first item
        chosen = items[0]

    track_id = chosen["id"]
    artist_ids = [a["id"] for a in chosen["artists"]]

    # fetch artist genres (batching: you can use sp.artists(list_of_ids))
    try:
        artists_info = sp.artists(artist_ids)
        genres_list = []
        for art in artists_info["artists"]:
            genres_list.extend(art.get("genres", []))
        # unique genres
        genres_unique = list(dict.fromkeys(genres_list))
    except Exception as e:
        print("Failed to get artist info:", e)
        genres_unique = []

    entry = {
        "title": qtitle,
        "artist": qartist,
        "track_id": track_id,
        "artist_ids": artist_ids,
        "artist_genres": genres_unique,
        "match_score": 1,  # heuristically set; could compute similarity
        "spotify_found": True
    }

    cache = pd.concat([cache, pd.DataFrame([entry])], ignore_index=True)
    results.append(entry)

    # persist incrementally to avoid losing progress on long runs
    save_cache()

    # be polite: avoid rate limits (adjust as needed)
    time.sleep(0.12)

# convert results to DataFrame and merge back to Billboard
res_df = pd.DataFrame(results)
# convert genres list to joined string
res_df["genre_join"] = res_df["artist_genres"].apply(lambda g: "; ".join(g) if isinstance(g, list) else None)

# merge back using normalized keys
billboard_df["song_key"] = billboard_df["song"].astype(str).str.lower().str.strip()
billboard_df["artist_key"] = billboard_df["artist"].astype(str).str.lower().str.strip() if "artist" in billboard_df.columns else ""
res_df["title_key"] = res_df["title"]
res_df["artist_key"] = res_df["artist"]

merged = billboard_df.merge(res_df[["title_key","artist_key","genre_join"]], how="left",
                            left_on=["song_key","artist_key"], right_on=["title_key","artist_key"])

merged = merged.drop(columns=["title_key","artist_key"])

merged.to_csv("data/hot100_with_spotify_genres.csv", index=False)
print("Saved merged file with genres (some may be NA).")


SyntaxError: invalid syntax (3829326670.py, line 1)

Based on this, can we attribute the missing genre-matching to pure-randomness? Is this sample of the billboard 100 chart even though we don't have every song representative? Is it okay to have missing values in this case? Basically, should we use this or should we just scratch the idea of trying to understand how the genres of charting music has changed over time?